## QDenseBatchnorm Test Notebook
***
QDenseBatchnorm should fold quantized Dense parameters into the the Batchnorm layer to reduce the LUT (look up table) utilization at HLS and Vivado FPGA Implementation time

In [1]:
import common as com
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Activation
from tensorflow.keras.regularizers import l1
from qkeras.qlayers import QDense, QActivation
from qkeras import QDenseBatchnorm
from qkeras.quantizers import quantized_bits, quantized_relu
import os
import hls4ml
from plot_roc import plot_roc

os.environ['PATH'] = '/opt/local/Xilinx/Vivado/2019.1/bin:' + os.environ['PATH']

2022-01-11 13:05:20.502795: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-11 13:05:20.502817: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/jmuhizi/batchnorm-fold-tutorials/hide_hls4ml/hls4ml/hls4ml/converters/__init__.py:16: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


In [2]:
def reference_model(bits=12, int_bits=0, relu_bits = 6):
    '''
    define the reference quantized model for the autoencoder, with fewer layers
    '''
    inputLayer = Input(shape=(128))
    kwargs = {
        'kernel_quantizer': quantized_bits(bits, int_bits, alpha=1),
        'bias_quantizer': quantized_bits(bits, int_bits, alpha=1),
        'kernel_initializer': 'lecun_uniform',
        'kernel_regularizer': l1(0) 
    }
    
    #declare encoder
    h = QDense(64, **kwargs)(inputLayer)
    h = BatchNormalization()(h)
    h = QActivation(activation=quantized_relu(relu_bits,0))(h)
    
    h = QDense(64, **kwargs)(h)
    h = BatchNormalization()(h)
    h = QActivation(activation=quantized_relu(relu_bits,0))(h)

    # h = QDense(64, **kwargs)(h)
    # h = BatchNormalization()(h)
    # h = QActivation(activation=quantized_relu(relu_bits,0))(h)

    # h = QDense(64, **kwargs)(h)
    # h = BatchNormalization()(h)
    # h = QActivation(activation=quantized_relu(relu_bits,0))(h)
    
    #declare latent layer
    h = h = QDense(8, **kwargs)(h)
    h = BatchNormalization()(h)
    h = QActivation(activation=quantized_relu(relu_bits,0))(h)
    
    #declare decoder
    h = QDense(64, **kwargs)(h)
    h = BatchNormalization()(h)
    h = QActivation(activation=quantized_relu(relu_bits,0))(h)
    
    h = h = QDense(64, **kwargs)(h)
    h = BatchNormalization()(h)
    h = QActivation(activation=quantized_relu(relu_bits,0))(h)

    # h = QDense(64, **kwargs)(inputLayer)
    # h = BatchNormalization()(h)
    # h = QActivation(activation=quantized_relu(relu_bits,0))(h)
    
    # h = QDense(64, **kwargs)(h)
    # h = BatchNormalization()(h)
    # QActivation(activation=quantized_relu(relu_bits,0))(h)
    
    #output layer
    h = QDense(128, **kwargs)(h)
    
    return Model(inputs=inputLayer, outputs=h)

    
def qdense_batchnorm_model(bits=12, int_bits=0, relu_bits = 6):
    '''
    define the densebatchnorm quantized model for the autoencoder, with fewer layers
    '''
    inputLayer = Input(shape=(128))
    kwargs = {
        'kernel_quantizer': quantized_bits(bits, int_bits, alpha=1),
        'bias_quantizer': quantized_bits(bits, int_bits, alpha=1),
        'kernel_initializer': 'lecun_uniform',
        'kernel_regularizer': l1(0) 
    }
    
    #declare encoder
    h = QDenseBatchnorm(64, **kwargs)(inputLayer)
    h = QActivation(activation=quantized_relu(relu_bits,0))(h)
    
    h = QDenseBatchnorm(64, **kwargs)(h)
    h = QActivation(activation=quantized_relu(relu_bits,0))(h)

    # h = QDenseBatchnorm(64, **kwargs)(h)
    # h = QActivation(activation=quantized_relu(relu_bits,0))(h)

    # h = QDenseBatchnorm(64, **kwargs)(h)
    # h = QActivation(activation=quantized_relu(relu_bits,0))(h)
    
    #declare latent layer
    h = QDenseBatchnorm(8, **kwargs)(h)
    h = QActivation(activation=quantized_relu(relu_bits))(h)
    
    #declare decoder
    h = QDenseBatchnorm(64, **kwargs)(h)
    h = QActivation(activation=quantized_relu(relu_bits,0))(h)

    h = QDenseBatchnorm(64, **kwargs)(h)
    h = QActivation(activation=quantized_relu(relu_bits,0))(h)

    # h = QDenseBatchnorm(64, **kwargs)(inputLayer)
    # h = QActivation(activation=quantized_relu(relu_bits,0))(h)

    # h = QDenseBatchnorm(64, **kwargs)(inputLayer)
    # h = QActivation(activation=quantized_relu(relu_bits,0))(h)
    
    #output layer
    h = QDense(128, **kwargs)(h)
    
    return Model(inputs=inputLayer, outputs=h)


### Create Train Data using ToyADMOS toy car dataset

In [3]:
param = com.yaml_load('data_gen_config.yml')
param = param["train"]
train_data_save_load_directory = "./train_time_data/train_data_inputs_{}_frames_{}_hops_{}_fft_{}_mels_{}_power_{}_downsample_{}.npy".format(
        param["model"]["input_dim"],param["feature"]["frames"], param["feature"]["hop_length"], 
            param["feature"]["n_fft"], param["feature"]["n_mels"], param["feature"]["power"],param["feature"]["downsample"],)
        
# if train_data available, load processed data in local directory without reprocessing wav files --saves time--
if os.path.exists(train_data_save_load_directory):
    print("Loading train_data from {}".format(train_data_save_load_directory))
    train_data = np.load(train_data_save_load_directory)
else:
    !python generate_train_data.py -c data_gen_config.yml
    train_data = np.load(train_data_save_load_directory)

Loading train_data from ./train_time_data/train_data_inputs_128_frames_5_hops_512_fft_1024_mels_128_power_2.0_downsample_True.npy


### train and test reference and qdense_batchnorm model

In [4]:
if not os.path.exists('./model/reference_model.h5'):
    ref_model = reference_model()
    ref_model.compile(loss='mean_squared_error', optimizer='adam')
    ref_model.summary()
    ref_model.fit(train_data, train_data, batch_size=512, epochs=20, shuffle=1,validation_split=0.1, verbose=1)
    ref_model.save('./model/reference_model.h5')
else:
    ref_model = com.load_model('./model/reference_model.h5')

2022-01-11 13:05:22.438337: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-11 13:05:22.438369: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-11 13:05:22.438394: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (correlator3.fnal.gov): /proc/driver/nvidia/version does not exist
2022-01-11 13:05:22.438579: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
if not os.path.exists('./model/qdbn_model.h5'):
    qdbn_model = qdense_batchnorm_model()
    qdbn_model.compile(loss='mean_squared_error', optimizer='adam')
    qdbn_model.summary()
    qdbn_model.fit(train_data, train_data, batch_size=512, epochs=20, shuffle=1,validation_split=0.1, verbose=1)
    qdbn_model.save('./model/qdbn_model.h5')
else:
    qdbn_model = com.load_model('./model/qdbn_model.h5')

### Compare QDenseBN (qdbn_model) and QDense->BN (ref_model) peformance

In [6]:
# if train_data available, load processed data in local directory without reprocessing wav files --saves time--
X_npy_dir =  './test_data/anomaly_detection/128input_test_data.npy'
y_npy_dir = './test_data/anomaly_detection/128input_test_data_ground_truths.npy'

In [7]:
plot_roc(ref_model, qdbn_model, X_npy_dir, y_npy_dir, output_dir = './')

using one quarter of provided dataset for roc plot


 29%|████████████▍                             | 181/614 [00:18<00:41, 10.55it/s]


 60%|█████████████████████████                 | 366/614 [00:35<00:23, 10.67it/s]


 91%|██████████████████████████████████████    | 556/614 [00:53<00:05, 11.21it/s]


 19%|███████▉                                  | 117/615 [00:11<00:47, 10.53it/s]


...

 22%|█████████▎                                | 136/615 [00:12<00:41, 11.49it/s]


 53%|██████████████████████▍                   | 328/615 [00:31<00:28, 10.10it/s]


 81%|██████████████████████████████████▏       | 501/615 [00:47<00:10, 10.67it/s]


100%|██████████████████████████████████████████████| 4/4 [03:56<00:00, 59.18s/it]
/home/jmuhizi/batchnorm-fold-tutorials/plot_roc.py:60: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


QDense vs QDenseBN plot saved in ./


### HLS4ML Translation of QDenseBN and QDense->BN models

In [8]:
hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'
hls_config = hls4ml.utils.config_from_keras_model(qdbn_model, granularity='name')
hls_config['Model']['ReuseFactor'] = 16384
hls_config['Model']['Strategy'] = 'Resource'
hls_config['Model']['Precision'] = 'ap_fixed<32,16>'
hls_config['LayerName']['input_2']['Precision'] = 'ap_fixed<8,8>'
for layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][layer]['Trace'] = True
    hls_config['LayerName'][layer]['accum_t'] = 'ap_fixed<32,16>'
    hls_config['LayerName'][layer]['ReuseFactor'] = 16384

backend='VivadoAccelerator'
clock_period=10
io_type='io_stream' 
interface='axi-stream'
cfg = hls4ml.converters.create_config(
            backend='VivadoAccelerator',
            part='xc7z020clg400-1',
            clock_period=10,
            io_type='io_stream')

cfg['HLSConfig'] = hls_config
cfg['InputData'] = 'test_data/anomaly_detection/test_bench/128input_test_data.npy'
cfg['OutputPredictions'] = 'test_data/anomaly_detection/test_bench/128input_test_data.npy'
cfg['KerasModel'] = qdbn_model
cfg['OutputDir'] = 'hls/qdbn_prj'

print("-----------------------------------")
#com.print_dict(cfg)
print("-----------------------------------")

qdbn_hls_model = hls4ml.converters.keras_to_hls(cfg)

qdbn_hls_model.compile()

Interpreting Model
Topology:
Layer name: input_2, layer type: Input
Layer name: q_dense_batchnorm, layer type: QDenseBatchnorm
Layer name: q_activation_5, layer type: QActivation
Layer name: q_dense_batchnorm_1, layer type: QDenseBatchnorm
Layer name: q_activation_6, layer type: QActivation
Layer name: q_dense_batchnorm_2, layer type: QDenseBatchnorm
Layer name: q_activation_7, layer type: QActivation
Layer name: q_dense_batchnorm_3, layer type: QDenseBatchnorm
Layer name: q_activation_8, layer type: QActivation
Layer name: q_dense_batchnorm_4, layer type: QDenseBatchnorm
Layer name: q_activation_9, layer type: QActivation
Layer name: q_dense_6, layer type: QDense
-----------------------------------
-----------------------------------
Interpreting Model
Topology:
Layer name: input_2, layer type: InputLayer, input shapes: [[None, 128]], output shape: [None, 128]
QDenseBatchnorm
Layer name: q_dense_batchnorm, layer type: QDenseBatchnorm, input shapes: [[None, 128]], output shape: [None, 

In [9]:
hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'
hls_config = hls4ml.utils.config_from_keras_model(ref_model, granularity='name')
hls_config['Model']['ReuseFactor'] = 16384
hls_config['Model']['Strategy'] = 'Resource'
hls_config['Model']['Precision'] = 'ap_fixed<32,16>'
hls_config['LayerName']['input_1']['Precision'] = 'ap_fixed<8,8>'
for layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][layer]['Trace'] = True
    hls_config['LayerName'][layer]['accum_t'] = 'ap_fixed<32,16>'
    hls_config['LayerName'][layer]['ReuseFactor'] = 16384

backend='VivadoAccelerator'
clock_period=10
io_type='io_stream' 
interface='axi-stream'
cfg = hls4ml.converters.create_config(
            backend='VivadoAccelerator',
            part='xc7z020clg400-1',
            clock_period=10,
            io_type='io_stream')

cfg['HLSConfig'] = hls_config
cfg['InputData'] = 'test_data/anomaly_detection/test_bench/128input_test_data.npy'
cfg['OutputPredictions'] = 'test_data/anomaly_detection/test_bench/128input_test_data.npy'
cfg['KerasModel'] = ref_model
cfg['OutputDir'] = 'hls/ref_prj'

print("-----------------------------------")
print(cfg)
print("-----------------------------------")

ref_hls_model = hls4ml.converters.keras_to_hls(cfg)

ref_hls_model.compile()

Interpreting Model
Topology:
Layer name: input_1, layer type: Input
Layer name: q_dense, layer type: QDense
Layer name: batch_normalization, layer type: BatchNormalization
Layer name: q_activation, layer type: QActivation
Layer name: q_dense_1, layer type: QDense
Layer name: batch_normalization_1, layer type: BatchNormalization
Layer name: q_activation_1, layer type: QActivation
Layer name: q_dense_2, layer type: QDense
Layer name: batch_normalization_2, layer type: BatchNormalization
Layer name: q_activation_2, layer type: QActivation
Layer name: q_dense_3, layer type: QDense
Layer name: batch_normalization_3, layer type: BatchNormalization
Layer name: q_activation_3, layer type: QActivation
Layer name: q_dense_4, layer type: QDense
Layer name: batch_normalization_4, layer type: BatchNormalization
Layer name: q_activation_4, layer type: QActivation
Layer name: q_dense_5, layer type: QDense
-----------------------------------
{'OutputDir': 'hls/ref_prj', 'ProjectName': 'myproject', 'Ba

### Compare performance of HLS (C++) QDenseBN vs QDense->BN

In [ ]:
plot_roc(ref_hls_model, qdbn_hls_model, X_npy_dir, y_npy_dir, output_dir = './hls_ref_vs_qdbn')

using one quarter of provided dataset for roc plot


 14%|██████▏                                    | 89/614 [00:14<01:27,  6.02it/s]

In [ ]:
ref_hls_model.build(reset=False, csim=True, cosim=True, validation=True, synth=True, vsynth=True, export=True)
hls4ml.report.read_vivado_report('hls/ref_prj')

In [ ]:
qdbn_hls_model.build(reset=False, csim=True, cosim=True, validation=True, synth=True, vsynth=True, export=True)
hls4ml.report.read_vivado_report('hls/qdbn_prj')